In [ ]:
### ZERO-SHOT CLASSIFICATION, TOPIC LABELLING ###

# TODOs: https://towardsdatascience.com/zero-shot-text-classification-with-hugging-face-7f533ba83cd6
# TODOs: Extract args, ids for labelling
# TODOs: Get Labels
# TODOs: NLI vs Semantic
# TODOs: Expand: Concept Net synonyms

In [7]:
import pandas as pd
import json

In [9]:
args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
len(args)

2509

In [4]:
ids_ = []
args_ = []

{'id': 't3_1egv4k', 'title': 'I think rape porn is despicable and that it should be made illegal in the U.S.', 'argument': {'argument': '', 'arg_keyphrases': [], 'arg_stance': ' '}, 'counter_tgt': {'counter': 'I dont know enough about the issue to comment on several of your points but for at least the bit about whether it encourages rape in real life statistics would make that seem unlikely. Id recommend reading that article to see why Im confident that the relationship between rape and internet access is not just correlation.', 'counter_keyphrases': ['real life statistics', 'issue to comment', 'real life', 'life statistics', 'statistics would make']}}


In [10]:
print(len(args_))
print(len(ids_))

2509
2509


In [13]:
args_

['I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes.',
 'My wife has recently gotten deeply involvedobsessed with the healing properties of essential oils. I am cursed with a common problem in that I am an extremely skeptical person. I cant help but look at this product and see a pyramid scheme that takes advantage of the gullible. All the shit my wife watches on these oils is carefully worded to make sure they dont make actual healing claims Im told this is because they aren

In [11]:
# Combined Label Set - CauseNet
# Experiment Title vs Argument
# Eval 1: Arguments

corpus = sorted(set(args_.tolist()))
corpus

AttributeError: 'list' object has no attribute 'tolist'

In [25]:
### COMMON SENSE TOPIC LABELS ###
import re

#TODOs: Prune and Add CauseNet Topics
cs_controversial_topics = [ln.strip() for ln in open("../../data/controversial_topics.txt")]
cs_debate_topics = [ln.strip() for ln in open("../../data/IBM_debator_topics.txt")]
cs_debate_topics_ = [ln.strip() for ln in open("../../data/IBM_debator_2.txt")]

topics = []
topics.extend(cs_controversial_topics)
topics.extend(cs_debate_topics)
topics.extend(cs_debate_topics_)

In [32]:
# Semantic-Search, Cosine Similarity
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = topics
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = sorted(set(args.titles.tolist()))
arg_topics = dict((k, []) for k in queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(1, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        #print(corpus[idx], "(Score: {:.4f})".format(score))
        arg_topics[query].append(corpus[idx])

In [33]:
arg_topics

{'. I am insensitive to the Gaming Industrys portrayal of women minorities and how it deals with other socially sensitive topics.': ['Sexism in video gaming'],
 '. I believe in same sex and traditional marriage but not other types of marriage': ['Same-sex marriage'],
 '. I believe the CBS show Elementary Is worth more praise and acclaim than the BBC version Sherlock': ['Criticism of the BBC'],
 'A Macbook is a better long term investment than a Windows computer.': ['Criticism of Windows Vista'],
 'A Mexican Border Fence Would Be an Ineffective Deterrent and a Colossal Waste of Taxpayer Money and Resources': ['Mexico–United States barrier'],
 'A Wired Mouse is better than a Wireless mouse.': ['Wireless'],
 'A banana is not a good method of portraying scale.': ['Principle'],
 'A comic book priced at . or . is disproportionately expensive to the amount of entertainment that can be garnered from it.': ['Predatory pricing'],
 'A congressional reboot would breathe new life into the legislati

In [45]:
fout = open("../../data/claim_topics.jsonl", "w")

for k, v in arg_topics.items():
    fout.write(json.dumps({
        "claim": k,
        "topic": v
    }))
    fout.write("\n")

In [26]:
# # Semantic-Search, Cosine Similarity
# from sentence_transformers import SentenceTransformer, util
# import torch

# embedder = SentenceTransformer('all-MiniLM-L6-v2')

# # Corpus with example sentences
# corpus = topics
# corpus_embeddings = embedder.encode(hate_corpus, convert_to_tensor=True)

# # Query sentences:
# queries = sorted(set(hate.hateSpeech.tolist()))
# hate_topics = dict((k, []) for k in queries)

# # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
# top_k = min(1, len(corpus))
# for query in queries:
#     query_embedding = embedder.encode(query, convert_to_tensor=True)

#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     for score, idx in zip(top_results[0], top_results[1]):
#         #print(corpus[idx], "(Score: {:.4f})".format(score))
#         hate_topics[query].append(corpus[idx])

In [27]:
# hate_topics

{'"islamophobia" is a way to take away our freedom of expression.': ['2003 invasion of Iraq'],
 '"l\'islamophobie" c\'est les musulmans qui veulent empêcher le blasphème.': ['Abortion'],
 '"l\'islamophobie" c\'est un moyen de nous retirer notre liberté d\'expression.': ['Affirmative action'],
 '#saville town looks like a city of Afghanistan or Pakistan more than an English City. What do you think?': ['African National Congress'],
 '#visitmymosque. But do not ask any questions about Muslim grooming gangs targeting white schoolgirls in Rotherham, Rochdale, Tyneside, Teeside etc.': ['Alberta separatism'],
 '#visitmymosque.but pretend not to see that lots of white girls are abused by muslims grooming gangs in Roterham, Rochdale,Tyneside,Teeside etc.': ['American Civil Liberties Union'],
 "(en réagissant à une attaque terroriste:) bien sûr encore un blond aux yeux bleus qui s'appelle édouard.": ['American Hunters and Shooters Association'],
 "(en réagissant à une attaque terroriste:) encore